In [1]:
#!/usr/bin/env python

# Copyright (c) 2019 Computer Vision Center (CVC) at the Universitat Autonoma de
# Barcelona (UAB).
#
# This work is licensed under the terms of the MIT license.
# For a copy, see <https://opensource.org/licenses/MIT>.

import glob
import os
import sys

try:
    sys.path.append(glob.glob('../carla/dist/carla-*%d.%d-%s.egg' % (
        sys.version_info.major,
        sys.version_info.minor,
        'win-amd64' if os.name == 'nt' else 'linux-x86_64'))[0])
except IndexError:
    pass

import carla

import random
import time

In [2]:
actor_list = []

# In this tutorial script, we are going to add a vehicle to the simulation
# and let it drive in autopilot. We will also create a camera attached to
# that vehicle, and save all the images generated by the camera to disk.

In [3]:
# 시뮬레이터에 요청을 보낼 클라이언트를 생성. 
# 여기서는 시뮬레이터가 포트 2000의 로컬 호스트에서 요청을 수신 가정.

client = carla.Client('localhost', 2000)
client.set_timeout(2.0)

In [4]:
# 클라이언트가 생기면 현재 실행 중인 월드를 검색할 수 있다.
world = client.get_world()

In [5]:
print(world)

World(id=1426253462908250787)


In [6]:
# world에는 시뮬레이션에 새로운 actors를 추가하는 데 사용할 수 있는 list blueprint가 포함되어 있다.
blueprint_library = world.get_blueprint_library()
print(blueprint_library)

[ActorBlueprint(id=static.prop.mesh,tags=[prop, mesh, static]), ActorBlueprint(id=static.prop.trashcan01,tags=[prop, trashcan01, static]), ActorBlueprint(id=static.prop.trashbag,tags=[trashbag, prop, static]), ActorBlueprint(id=static.prop.trafficwarning,tags=[trafficwarning, prop, static]), ActorBlueprint(id=static.prop.trafficcone02,tags=[trafficcone02, prop, static]), ActorBlueprint(id=static.prop.trafficcone01,tags=[prop, trafficcone01, static]), ActorBlueprint(id=static.prop.table,tags=[table, prop, static]), ActorBlueprint(id=static.prop.streetbarrier,tags=[streetbarrier, prop, static]), ActorBlueprint(id=static.prop.shoppingtrolley,tags=[shoppingtrolley, prop, static]), ActorBlueprint(id=static.prop.shoppingbag,tags=[shoppingbag, prop, static]), ActorBlueprint(id=static.prop.shoppingcart,tags=[shoppingcart, prop, static]), ActorBlueprint(id=static.prop.plasticchair,tags=[prop, plasticchair, static]), ActorBlueprint(id=static.prop.plantpot08,tags=[plantpot08, prop, static]), Acto

In [7]:
# 이제 '차량' 유형의 blueprint를 모두 필터링하고 무작위로 하나를 선택해 보겠습니다.
bp = random.choice(blueprint_library.filter('vehicle'))
print(bp)

ActorBlueprint(id=vehicle.mercedes.coupe_2020,tags=[coupe_2020, mercedes, vehicle])


In [8]:
# Blueprint에는 차량의 인스턴스를 정의하는 속성 목록이 포함되어 있으며, 이를 읽고 일부를 수정할 수 있습니다. 
# 예를 들어, 그것의 색깔을 무작위화해 봅시다.
if bp.has_attribute('color'):
    color = random.choice(bp.get_attribute('color').recommended_values)
    bp.set_attribute('color', color)

print(bp)
print(bp.get_attribute('color'))

ActorBlueprint(id=vehicle.mercedes.coupe_2020,tags=[coupe_2020, mercedes, vehicle])
ActorAttribute(id=color,type=Color,value=Color(0,21,81,255))


In [9]:
# 이제 우리는 차량에 초기 변형을 주어야 한다. 
# 우리는 지도의 list of recommended spawn points에서 무작위 변환을 선택한다.
transform = random.choice(world.get_map().get_spawn_points())

# start_location = carla.Location(x=0, y=13, z=2.0)
start_location = carla.Location(x=-65.0, y=13, z=2.0)
start_rotation = carla.Rotation(pitch=0.0, yaw=180, roll=0.0)

start_transform = carla.Transform(start_location, start_rotation)
print(start_transform)

Transform(Location(x=-65.000000, y=13.000000, z=2.000000), Rotation(pitch=0.000000, yaw=180.000000, roll=0.000000))


In [10]:
# world에 차량을 spawn 하라고 하자.
vehicle = world.spawn_actor(bp, start_transform)
print(vehicle)  # 빨간색 소방차(차종 랜덤)가 추가됨을 확인함.

Actor(id=24, type=vehicle.mercedes.coupe_2020)


In [34]:
waypoint = world.get_map().get_waypoint(vehicle.get_location(),project_to_road=True, lane_type=(carla.LaneType.Driving | carla.LaneType.Shoulder | carla.LaneType.Sidewalk))
print("Current lane type: " + str(waypoint.lane_type))
# Check current lane change allowed
print("Current Lane change:  " + str(waypoint.lane_change))
# Left and Right lane markings
print("L lane marking type: " + str(waypoint.left_lane_marking.type))
print("L lane marking change: " + str(waypoint.left_lane_marking.lane_change))
print("R lane marking type: " + str(waypoint.right_lane_marking.type))
print("R lane marking change: " + str(waypoint.right_lane_marking.lane_change))

# landmark = waypoint.get_landmarks_of_type(30, stop_at_junction=False)
# print(landmark)
vehicle.set_autopilot(False)
print(vehicle.get_traffic_light())

Current lane type: Driving
Current Lane change:  Left
L lane marking type: Broken
L lane marking change: Both
R lane marking type: Solid
R lane marking change: NONE
Actor(id=9, type=traffic.traffic_light)


In [39]:
# start_trans = carla.Transform(Location(x=-65.0, y=13.0, z=2.0), \
#                              Rotation(pitch=-0.0, yaw=-180, roll=0.0))
# way_point1 = carla.Transform(Location(x=-85.0, y=13.0, z=0.0), \
#                             Rotation(pitch=-0.0, yaw=-180.0, roll=0.0))
# way_point2 = carla.Transform(Location(x=-103.5, y=-2, z=-0.0), \
#                              Rotation(pitch=-0.3, yaw=-90.0, roll=0.0))
# way_point3 = carla.Transform(Location(x=-69.5, y=-58.3, z=-0.0), \
#                               Rotation(pitch=-0.3, yaw=0.0, roll=0.0))
# way_point4 = carla.Transform(Location(x=-52.3, y=-4.3, z=-0.0), \
#                              Rotation(pitch=-0.3, yaw=90.0, roll=0.0))

# # ================================ go to start point ==================================== #
# stt = carla.Transform(carla.Location(x=-29.0, y=13.0, z=2.0), \
#                              carla.Rotation(pitch=-0.0, yaw=-180, roll=0.0))
# vehicle.set_transform(stt)
# # ======================================================================================= #

print(vehicle.get_transform()) 
# ==============================================================================
start_location.x += 20
# start_rotation.yaw += 90
tmp_transform = carla.Transform(start_location, start_rotation)
vehicle.set_transform(tmp_transform)
# ==============================================================================

Transform(Location(x=-28.000069, y=12.999969, z=-0.004697), Rotation(pitch=0.010348, yaw=179.999969, roll=0.003763))


In [179]:
# 우리가 만든 actors는 destroy 함수를 호출하지 않는 한, 파괴되지 않는다.
# destroy 함수를 호출하지 않으면, Python 스크립트를 종료한 이후에도 시뮬레이션에 남는 것에 주의
# 이런 이유로, 우리가 만든 actors를 한번에 destroy 할 수 있도록 list로 관리하자 
actor_list.append(vehicle)
print('created %s' % vehicle.type_id)

created vehicle.mini.cooper_s


In [180]:
# Let's put the vehicle to drive around.
vehicle.set_autopilot(True)

In [96]:
# 차량에 부착된 "depth camera"를 추가해 보겠다. 
# 여기서 설명하는 transform은 이제 차량과 관련이 있다.
camera_bp = blueprint_library.find('sensor.camera.depth')
camera_transform = carla.Transform(carla.Location(x=1.5, z=2.4))
camera = world.spawn_actor(camera_bp, camera_transform, attach_to=vehicle)
actor_list.append(camera)
print('created %s' % camera.type_id)

print(actor_list) # there is 1 car & 1 camera

created sensor.camera.depth
[<carla.libcarla.ServerSideSensor object at 0x7f14443f3240>, <carla.libcarla.Vehicle object at 0x7f14443f30e0>, <carla.libcarla.Vehicle object at 0x7f14443f30e0>, <carla.libcarla.ServerSideSensor object at 0x7f14361ffbe0>]


In [97]:
# 센서가 이미지를 수신할 때마다 호출될 function을 등록
# 이 예에서는 이미지를 픽셀을 그레이 스케일로 변환하여 디스크에 저장합니다.

cc = carla.ColorConverter.LogarithmicDepth
camera.listen(lambda image: image.save_to_disk('_out/%06d.png' % image.frame, cc))

In [15]:
# Oh wait, I don't like the location we gave to the vehicle, I'm going
# to move it a bit forward.
location = vehicle.get_location()
location.x += 20                # +x : 차량의 정면 방향, +y : 차량의 우측 방향, +z : 하늘 방향
vehicle.set_location(location)
print('moved vehicle to %s' % location)

moved vehicle to Location(x=-19.275845, y=27.963459, z=-0.019143)


In [30]:
# 차량을 몇 대 더 추가해 보자.
transform.location += carla.Location(x=40, y=-3.2)
transform.rotation.yaw = -180.0
for _ in range(0, 10):
    transform.location.x += 8.0

    bp = random.choice(blueprint_library.filter('vehicle'))

    # 이번에는 try_spawn_actor를 사용합니다. 
    # 장소(spot)가 이미 다른 객체(object)에 의해 점유된(occupied) 경우 함수는 None을 반환합니다.
    npc = world.try_spawn_actor(bp, transform)
    if npc is not None:
        actor_list.append(npc)
        npc.set_autopilot(True)
        print('created %s' % npc.type_id)

time.sleep(5)

created vehicle.nissan.patrol
created vehicle.carlamotors.carlacola
created vehicle.audi.etron
created vehicle.lincoln.mkz_2020
created vehicle.dodge.charger_police_2020
created vehicle.lincoln.mkz_2020
created vehicle.kawasaki.ninja
created vehicle.vespa.zx125
created vehicle.yamaha.yzf
created vehicle.bmw.grandtourer


In [182]:
print('destroying actors')
camera.destroy()
client.apply_batch([carla.command.DestroyActor(x) for x in actor_list])
print('done.')

destroying actors
done.


---

In [101]:
# tmp
import cv2
import numpy as np

client = carla.Client("192.168.0.4", 2000)
print("carla connected!")
client.set_timeout(2.0)

world = client.get_world()
blueprint_library = world.get_blueprint_library()
camera_bp = blueprint_library.find('sensor.camera.rgb')
camera_transform = carla.Transform(carla.Location(x=1.5, z=2.4))
vehicle = world.get_actors().filter('vehicle.*')[0]
camera = world.spawn_actor(camera_bp, camera_transform, attach_to=vehicle)
print("camera created!", camera)
cc = carla.ColorConverter.Raw
# camera.listen(lambda image: cv2.imshow(np.frombuffer(image.raw_data, dtype=np.dtype("uint8"))))
camera.listen(lambda image: image.save_to_disk('_out/%06d.png' % image.frame, cc))

carla connected!
camera created! Actor(id=79, type=sensor.camera.rgb)


In [104]:
camera.destroy()

RuntimeError: trying to operate on a destroyed actor; an actor's function was called, but the actor is already destroyed.

---

In [165]:
# pedestrian code

world.set_pedestrians_seed(1235)
ped_bp = random.choice(world.get_blueprint_library().filter("walker.pedestrian.*"))

ped_start_location = carla.Location(x=-34.3, y=2.7, z=0.2)
ped_start_rotation = carla.Rotation(pitch=-0.3, yaw=-180.0, roll=0.0)
start_trans = carla.Transform(ped_start_location, ped_start_rotation)

ped_dst_location = carla.Location(x=-59.3, y=2.7) #z=0.134872
ped_dst_rotation = carla.Rotation(pitch=-0.3, yaw=-180.0, roll=0.0)
dst_trans = carla.Transform(ped_dst_location, ped_dst_rotation)

ped = world.spawn_actor(ped_bp, start_trans)
walker_controller_bp = world.get_blueprint_library().find('controller.ai.walker')
controller = world.spawn_actor(walker_controller_bp, carla.Transform(), ped)
controller.start()
controller.set_max_speed(1.7)

# adding an actor to an actor list
actor_list.append(ped)
actor_list.append(controller)

while True:
    x = ped.get_location().x
    y = ped.get_location().y        # restrict Unexpected movement

    if x < -61.0 or y < 0.5:
        controller.go_to_location(ped_start_location)
    elif x > -32.0 or y < 0.5:
        controller.go_to_location(ped_dst_location)


KeyboardInterrupt: 

In [167]:
ped.destroy()
controller.destroy()

False